In [1]:
import json
import pandas as pd
import pickle
import numpy as np

In [2]:
# Change this function to select a different paper set
def read_metadata(path):
    df = pd.read_csv(path)
    df = df.astype({'abstract': 'str'})
    df = df[df['abstract'] != 'Unknown']
    df = df[df['abstract'] != 'nan']
    df = df[df['language'] == 'en']
    #df = df[df['publish_time'].str[:4] == '2020']
    df = df.reset_index(drop=True)
    #df = df[:1000]
    return df

In [3]:
load_from_pickle = True
#EMBEDDINGS_PATH = 'data/embeddings_top5k.pkl'
#EMBEDDINGS_PATH = 'data/embeddings_2020.pkl'
#EMBEDDINGS_PATH = 'data/embeddings_2020_top1k.pkl'
#EMBEDDINGS_PATH = 'data/embeddings_sharp_top1k.pkl'
EMBEDDINGS_PATH = 'data/embeddings_subtopics_sharp_all.pkl'
#METADATA_PATH = 'data/metadata_new_new.csv'
#METADATA_PATH = 'data/metadata_sharp.csv'
METADATA_PATH = 'data/metadata_subtopics_sharp.csv'

df_meta = read_metadata(METADATA_PATH)
abstracts = [a for a in df_meta['abstract']]
titles = df_meta['title'].reset_index(drop=True)
cord_uids = df_meta['cord_uid']

if load_from_pickle:
    with open(EMBEDDINGS_PATH, 'rb') as file:
        embeddings = pickle.load(file)

else:
    model = SentenceTransformer(MODEL_PATH)
    embeddings = model.encode(abstracts, show_progress_bar=True)
    with open(EMBEDDINGS_PATH, 'wb') as file:
        pickle.dump(embeddings, file)
        
embeddings_array = np.array(embeddings)

In [4]:
title_to_cord_uid = {}
for paper in df_meta.iterrows():
    series = paper[1]
    title_to_cord_uid.update({series['title']: series['cord_uid']})

In [5]:
n_topics = 8
n_subtopics = 8

for t in range(n_topics):
    for s in range(n_subtopics):
        filename = 'data/json_old/data_' + repr(t) + '_' + repr(s) + '.json'
        data = json.load(open(filename, 'rb'))
        nodes = data['nodes']
        for node in nodes:
            cord_uid = title_to_cord_uid[node['label']]
            node.update({'cord_uid': cord_uid})
            
        data['nodes'] = nodes
        # Save JSON
        filename = 'data/json/data_' + repr(t) + '_' + repr(s) + '.json'
        with open(filename, 'w') as outfile:
            json.dump(data, outfile)

The end.